<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

2023-03-20 20:07:56.550783: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Version:  2.11.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [3]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [4]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [5]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [6]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [7]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [8]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [9]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [10]:
!rm -rf logs

In [11]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10


2023-03-20 20:10:23.840808: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401
2023-03-20 20:10:27.106731: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7fd8624806b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-03-20 20:10:27.106770: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 2070 SUPER, Compute Capability 7.5
2023-03-20 20:10:27.114377: W tensorflow/compiler/xla/service/gpu/nvptx_helper.cc:56] Can't find libdevice directory ${CUDA_DIR}/nvvm/libdevice. This may result in compilation or runtime failures, if the program we try to run uses routines from libdevice.
Searched for CUDA in the following directories:
  ./cuda_sdk_lib
  /usr/local/cuda-11.2
  /usr/local/cuda
  .
You can choose the search directory by setting xla_gpu_cuda_data_dir in HloModule's DebugOptions.  For most apps, setting the environment variable XLA_FLAGS=--xla

26/26 [==============================] - 10s 59ms/step - loss: 0.6791 - accuracy: 0.5766 - val_loss: 0.7084 - val_accuracy: 0.4780
Epoch 2/10
26/26 [==============================] - 1s 25ms/step - loss: 0.6067 - accuracy: 0.6849 - val_loss: 0.5796 - val_accuracy: 0.6488
Epoch 3/10
26/26 [==============================] - 1s 21ms/step - loss: 0.5643 - accuracy: 0.7214 - val_loss: 0.5109 - val_accuracy: 0.8537
Epoch 4/10
26/26 [==============================] - 1s 22ms/step - loss: 0.5108 - accuracy: 0.7640 - val_loss: 0.5357 - val_accuracy: 0.6780
Epoch 5/10
26/26 [==============================] - 1s 20ms/step - loss: 0.4461 - accuracy: 0.8224 - val_loss: 0.4057 - val_accuracy: 0.8732
Epoch 6/10
26/26 [==============================] - 1s 23ms/step - loss: 0.3833 - accuracy: 0.8723 - val_loss: 0.3735 - val_accuracy: 0.8585
Epoch 7/10
26/26 [==============================] - 1s 22ms/step - loss: 0.3191 - accuracy: 0.8820 - val_loss: 0.3007 - val_accuracy: 0.9220
Epoch 8/10
26/26 [=====

In [12]:
%tensorboard --logdir logs

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 1: saving model to weights.01-0.69.h5
26/26 - 1s - loss: 0.6784 - accuracy: 0.5584 - val_loss: 0.6917 - val_accuracy: 0.4341 - 1s/epoch - 55ms/step
Epoch 2/5

Epoch 2: saving model to weights.02-0.63.h5
26/26 - 1s - loss: 0.6489 - accuracy: 0.6217 - val_loss: 0.6313 - val_accuracy: 0.8195 - 741ms/epoch - 28ms/step
Epoch 3/5

Epoch 3: saving model to weights.03-0.58.h5
26/26 - 1s - loss: 0.6035 - accuracy: 0.7640 - val_loss: 0.5788 - val_accuracy: 0.8293 - 755ms/epoch - 29ms/step
Epoch 4/5

Epoch 4: saving model to weights.04-0.52.h5
26/26 - 1s - loss: 0.5544 - accuracy: 0.7591 - val_loss: 0.5166 - val_accuracy: 0.8537 - 786ms/epoch - 30ms/step
Epoch 5/5

Epoch 5: saving model to weights.05-0.44.h5
26/26 - 1s - loss: 0.5189 - accuracy: 0.7543 - val_loss: 0.4445 - val_accuracy: 0.8878 - 745ms/epoch - 29ms/step


In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 1: saving model to saved_model


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


26/26 - 2s - loss: 0.6751 - accuracy: 0.5633 - val_loss: 0.6441 - val_accuracy: 0.7024 - 2s/epoch - 82ms/step


In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 1: saving model to model.h5
26/26 - 2s - loss: 0.6694 - accuracy: 0.5925 - val_loss: 0.6279 - val_accuracy: 0.7854 - 2s/epoch - 61ms/step
Epoch 2/2

Epoch 2: saving model to model.h5
26/26 - 1s - loss: 0.5835 - accuracy: 0.7567 - val_loss: 0.6074 - val_accuracy: 0.5902 - 717ms/epoch - 28ms/step


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 1s - loss: 0.6663 - accuracy: 0.6180 - val_loss: 0.6412 - val_accuracy: 0.7171 - 1s/epoch - 54ms/step
Epoch 2/50
26/26 - 1s - loss: 0.5950 - accuracy: 0.7251 - val_loss: 0.5531 - val_accuracy: 0.7805 - 623ms/epoch - 24ms/step
Epoch 3/50
26/26 - 1s - loss: 0.5252 - accuracy: 0.7470 - val_loss: 0.4857 - val_accuracy: 0.8390 - 636ms/epoch - 24ms/step
Epoch 4/50
26/26 - 1s - loss: 0.4522 - accuracy: 0.8114 - val_loss: 0.3977 - val_accuracy: 0.8341 - 665ms/epoch - 26ms/step
Epoch 5/50
26/26 - 1s - loss: 0.4016 - accuracy: 0.8333 - val_loss: 0.3448 - val_accuracy: 0.8780 - 672ms/epoch - 26ms/step
Epoch 6/50
26/26 - 1s - loss: 0.3135 - accuracy: 0.8966 - val_loss: 0.3127 - val_accuracy: 0.9024 - 603ms/epoch - 23ms/step
Epoch 7/50
26/26 - 1s - loss: 0.2830 - accuracy: 0.9002 - val_loss: 0.2448 - val_accuracy: 0.8976 - 613ms/epoch - 24ms/step
Epoch 8/50
26/26 - 1s - loss: 0.2118 - accuracy: 0.9465 - val_loss: 0.2164 - val_accuracy: 0.9268 - 665ms/epoch - 26ms/step
Epoch 9/50


## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [17]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 1s 28ms/step - loss: 0.6541 - accuracy: 0.6217 - val_loss: 0.6787 - val_accuracy: 0.5220
Epoch 2/5
26/26 [==============================] - 1s 19ms/step - loss: 0.6177 - accuracy: 0.6703 - val_loss: 0.5666 - val_accuracy: 0.7902
Epoch 3/5
26/26 [==============================] - 1s 20ms/step - loss: 0.5379 - accuracy: 0.7482 - val_loss: 0.4758 - val_accuracy: 0.7805
Epoch 4/5
26/26 [==============================] - 1s 19ms/step - loss: 0.4791 - accuracy: 0.7968 - val_loss: 0.4119 - val_accuracy: 0.8439
Epoch 5/5
26/26 [==============================] - 1s 20ms/step - loss: 0.4135 - accuracy: 0.8260 - val_loss: 0.4243 - val_accuracy: 0.8146


In [18]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.621655,0.654131,0.521951,0.678745
1,1,0.670316,0.617722,0.790244,0.566583
2,2,0.748175,0.537947,0.780488,0.475765
3,3,0.796837,0.479101,0.843902,0.411930
4,4,0.826034,0.413545,0.814634,0.424312


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [19]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 1: LearningRateScheduler setting learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 1s 30ms/step - loss: 0.6765 - accuracy: 0.5560 - val_loss: 0.6673 - val_accuracy: 0.6195 - lr: 0.0050

Epoch 2: LearningRateScheduler setting learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 1s 20ms/step - loss: 0.6513 - accuracy: 0.6302 - val_loss: 0.6572 - val_accuracy: 0.6098 - lr: 0.0025

Epoch 3: LearningRateScheduler setting learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 1s 20ms/step - loss: 0.6401 - accuracy: 0.7032 - val_loss: 0.6557 - val_accuracy: 0.5659 - lr: 0.0012

Epoch 4: LearningRateScheduler setting learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 1s 20ms/step - loss: 0.6338 - accuracy: 0.6752 - val_loss: 0.6500 - val_accuracy: 0.6146 - lr: 6.2500e-04

Epoch 5: LearningRateScheduler setting learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 1s 

In [ ]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [20]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 1s 27ms/step - loss: 0.6688 - accuracy: 0.6071 - val_loss: 0.6431 - val_accuracy: 0.7561 - lr: 0.0100
Epoch 2/50
26/26 [==============================] - 1s 20ms/step - loss: 0.6220 - accuracy: 0.6837 - val_loss: 0.5965 - val_accuracy: 0.8000 - lr: 0.0100
Epoch 3/50
26/26 [==============================] - 1s 20ms/step - loss: 0.5878 - accuracy: 0.7117 - val_loss: 0.5457 - val_accuracy: 0.7220 - lr: 0.0100
Epoch 4/50
23/26 [=========================>....] - ETA: 0s - loss: 0.5208 - accuracy: 0.7649
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.
26/26 [==============================] - 1s 20ms/step - loss: 0.5190 - accuracy: 0.7664 - val_loss: 0.6126 - val_accuracy: 0.6000 - lr: 0.0100
Epoch 5/50
26/26 [==============================] - 1s 19ms/step - loss: 0.4560 - accuracy: 0.8248 - val_loss: 0.4535 - val_accuracy: 0.8585 - lr: 0.0020
Epoch 6/50
26/26 [==============================] - 1s 20ms/step - loss

In [ ]:
%tensorboard --logdir log_dir